# Logbook Metagenomics Project Salinity Stress
## abstract
## Introduction
## Workflow / Pipeline


## Logs

**Date: 5-03-2024**  

**Goals**  
- Seek tools needed for processing and analysing our data

**Results**  



I was able to find a wikipedia page giving a nive overview of all the steps involved in a metagenomics analysis. [metagenomics](https://en.wikipedia.org/wiki/Metagenomics). For us the main interest is in the non-assembled analysis. Because unreliable reads should not be assambled and we don't want homunculi genomes that we then try to find the organisms for. We want to use every read as a reference to find our potential organisms. This is called classification for which i have found atleast 

This was a good reference project [here](https://pmc.ncbi.nlm.nih.gov/articles/PMC8086013/)
I also found nanoclass2 which can taxonomic classify using a bunch of tools, though it seems this is tool does everything for you, which might not be something for us [here](https://ndombrowski.github.io/NanoClass2/source/run_nanoclass.html)

Kraken2, Kaiju, MegaBLAST, RDP,   ??> IDTAXA, [SPINGO](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-015-0747-1)

https://github.com/lsbnb/MetaSquare


A big question is which classifiers should we use to guarentee that the classified organisms are accurate. Because we don't know what organisms to expect and want to do a broad as possible search, a combination of these tools would be good choice.

Porechop for trimming adapters 
Filtlong or nanofilt for filtering reads based on quality.
Centrifuge, Kraken2, RDP and MegaBLAST for Taxonimy classification.


**Refelction**  
There is a large variety of tools and directions a metagenommics project can go. I am having a hard time creating something concreate, Ronald is hinting, sortof, but i am only more confused on what would be the best direction for our project. I know for sure classifing will get us a good way. another potential direction might be comparitive metagenomics.

**Next Steps**  
- Testing Trimming tools  

---

**Date: 11-03-2025**  

**Goals**  
- Setting up Kaiju in environment

**Results**  
After having a chat with Martijn, we have come to learn that our data has already been preprocessed. This means that trimming and removing adapters/barcodes is not nessisary anymore, for Martijn has already done this. Some waisted time here looking into preprocessing of data.  
The next step to processing our data will be to classify them and see what organisms exists in our samples. I already collected a few tools that could do this, but Martijn noted that Kraken2 and Kaiju are our best choices. Centrifuge was the predicessor of Kraken2, and after looking a bit better RDP and SPINGO are outdated begin between 5-10 years old and not being activaly maintained.

We also have the option to do functional annotation analaysis to see what proteins/pathways are pressent and analyse what their influence might be, but this is not relevent for awnsering our research objective. So we will omit this branch of metagenomics for future research and focus on making a good pipeline and interpreting our results. In note for future, if we can't identify a significant number of organisms, we could assemble genomes, but again this should not be nessisary.



Kaiju can easilly be installed via conda, after installing an index needs to be build or en pre-build index needs to be downloaded. The best course of action is to use the pre-build indexes, we don't have access to any specific databases/sets so a generic one is fine. The nr database containing a subset of the NCBI BLAST-rn database is a good coverage for Archaa, bacteria and viruses. Additionally for us fungi are interessting because of known interactions between plants and fungi supporting plant growth. So the fungi database or maybe better the nr_euk database which is like nr, but also includes fungi and microbial eukaryotes.

For now i will try and download the nr_euk and the fungi database index, but martijn said he will make them availible on the network aswell.


**Reflection**  
After thinking of our steps to be taken, run them by our teachers for some validation, this can save us some effort.

**Next steps**  
- Running Kaiju on data
- Setting up Snakemake script
- Adding Kaiju command to Snakemake

---

**Date: 12-03-2025**  

**Goals**  
- Running Kaiju on test data
- Starting basic snakemake pipline with config

**Results**  
Kaiju takes 3 atleast parameters, the .fmi which is the index file, a nodes.dmp file (also from the index) and the fastq file (which can be paired end aswell). Besides that a -z can be added for multithreading, kaiju-mulit can be used to supply a list of fastq files, which is what i want to use.

I added the basis for snakemake and a snakemake config. I also discovered that kaiju-multi is not part of the conda packages. So unless i want to install i seperatly i need to make do with just normal kaiju. Snakemake should be able to handel some of the parallel processing that kaiju would have otherwise done so it should be fine. 

```bash
kaiju -t /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/nodes.dmp -f /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/kaiju_db_fungi.fmi -i /commons/Themas/Thema07/metagenomics/zoutstress/16S/zout_20_30cm_rep2/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq.gz -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kaiju/FAY08769_pass_barcode10_85e26567_1cef8301_3.out
```

I Tried running a single instance of kaiju via the cli, but i would seem kaiju is having trouble recognising the file type. I tries two different fastq files to see if i might have been a problem with just the one file, but it appears to presist. Not really sure how to fix this, ill have to ask some advice.

```bash
kaiju -t /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/nodes.dmp -f /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/kaiju_db_fungi.fmi -i /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/16s/zout_20_30cm_rep2/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kaiju/FAY08769_pass_barcode10_85e26567_1cef8301_3.out
```
running it on the test data seems to work. These files have been decompressed.

```bash
kaiju -t /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/nodes.dmp -f /students/2024-2025/Thema07/metagenomics/zoutstress/kaiju_index/kaiju_db_fungi/kaiju_db_fungi.fmi -i /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq.gz -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kaiju/FAY08769_pass_barcode10_85e26567_1cef8301_3.out
```

After trying the same command on a recompressed testdata file the same error occures."Auto-detection of file type for file /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/FAY08769_pass_barcode10_85e26567_1cef8301_3.fastq.gz failed."

So even though in the examples of kaiju compressed fastq files are used. I seem to not be able to use them.



**Reflection**  
I definitly need to exercise more with snakemake i am having trouble writing the nessisary rules for running kaiju.

**Next steps**  
- Continue developing snakemake rules
- Run multi instance Kaiju

---


**Date: 14-03-2024**  

**Goals**
- Concatting data
- Running Kaiju on data
- Converting to Krona

**Results**  
I gained some more insights on our data from ronald. The minIOn creates a whole bunch of fastq files, but they all belong to the same sample. Meaning that all the fastq files can be concatted instead of being treated seperatly. I wrote a small bash command to handle the concatination of all the  fastq files. This will have to be incorperated into snakemake later. For now we have 8 fastq files that makes running kaiju a lot simpler.

```bash
cd /commons/Themas/Thema07/metagenomics/zoutstress/
data_dir="/students/2024-2025/Thema07/metagenomics/zoutstress/data/"
find . -maxdepth 2 -mindepth 2 -type d | parallel "mkdir -vpm 770 ${data_dir}{//}; cat {}/* > ${data_dir}{}.fastq.gz"
```



**Reflection**  

**Next Steps**  

 

**Date: xx-xx-xxxx**  
**Goals:**  
**Results**  
- What
- Why
- How
**Reflection**  
**Next Steps**  